# 9.4 확장된 합성곱 신경망 모델 클래스

## 9.4.1 기반 클래스 파일 실행시키기

In [1]:
%run ../chap08/cnn_reg_model.ipynb

## 9.4.2 CnnExtModel 클래스 선언

In [2]:
class CnnExtModel(CnnRegModel):
    macros = {} # 메서드 정의 바깥에서 선언되기 때문에 객체 변수가 아닌 클래스 변수,
                # custom 계층에서 처리할 매크로 정의 내용을 클래스 차원에서 관리하는 역할
        
    def __init__(self, name, dataset, hconfigs, show_maps=False,
                l2_decay=0, l1_decay=0, dump_structure=False):
        self.dump_structure = dump_structure # 구조 출력 여부를 지정, 파라미터 생성 메서드인 alloc_layer_param() 처리과정에도 영향
        self.layer_index = 0 # 중간 출력 생성 시 그 형태와 내용을 결정하는 값
        self.layer_depth = 0 # //
        self.param_count = 0 # 파라미터 수, dump_structure 값이 True이면 출력
        super(CnnExtModel, self).__init__(name, dataset, hconfigs, show_maps,
                                          l2_decay, l1_decay)
        if self.dump_structure:
            print('Total parameter count: {}'.format(self.param_count))

## 9.4.3 계층을 위한 파라미터 생성 메서드 정의

In [7]:
def cnn_ext_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)
    
    if layer_type in ['serial', 'parallel', 'loop', 'add', 'custom']:
        if self.dump_structure: 
            dump_str = layer_type # 계층 이름을 출력할 정보로 챙김
            if layer_type == 'custom':
                name = get_conf_param(hconfig, 'name') # custom 계층에 한해 매크로 내용으로 대체되어 표면상 사라지게 될
                                                       # 사용자 정의 계층이름을 출력 정보에 추가
                dump_str += ' ' + name
            print('{:>{width}}{}'.format('', dump_str, # 들여쓰기 하며 준비된 정보 출력(:>와 ''는 무엇인지 등 formating 순서???)
                width=self.layer_depth*2))
        self.layer_depth += 1 # 들여쓰기는 복합 구조 내부의 서브 신경망 구조들에 대해 메서드가 호출될 때 출력 형태를 알아보기 쉽게 함
    pm, output_shape = super(CnnExtModel, self).alloc_layer_param(input_shape, hconfig)
    
    if layer_type in ['serial', 'parallel', 'loop', 'add', 'custom']: # 다섯 가지 복합 계층에 대해서는 더이 상의 출력 없이 깊이 값만 복구
        self.layer_depth -= 1;
    elif self.dump_structure:
        self.layer_index += 1
        pm_str = '';
        if layer_type == 'full':
            ph, pw = pm['w'].shape
            pm_count = np.prod(pm['w'].shape) + pm['b'].shape[0]
            self.param_count += pm_count
            pm_str = ' pm:{}x{}+{}={}'.format(ph, pw, pm['b'].shape[0], pm_count)
        elif layer_type == 'conv':
            kh, kw, xchn, ychn = pm['k'].shape
            pm_count = np.prod(pm['k'].shape) + pm['b'].shape[0]
            self.param_count += pm_count
            pm_str = ' pm:{}x{}x{}x{}+{}={}'.format(kh, kw, xchn, ychn,
                                                   pm['b'].shape[0], pm_count)
        print('{:>{width}}{}: {}, {}=>{}{}'.
             format('', self.layer_index, layer_type, input_shape, output_shape, pm_str, width=self.layer_depth*2))
    return pm, output_shape

CnnExtModel.alloc_layer_param = cnn_ext_alloc_layer_param

## 9.4.4 병렬 계층 지원을 위한 세 개의 메서드 정의

In [ ]:
def cnn_ext_alloc_parallel_layer(self, input_shape, hconfig):
    pm_hiddens = []
    output_shape = None 
    
    if not isinstance(hconfig[1], dict): hconfig.insert(1, {}) # hconfig[2:] 접근에 문제를 일으키지 않게 함
        
    for bconfig in hconfig[2:]: # 각 분기에 대하여
        bpm, bshape = self.alloc_layer_param(input_shape, bconfig)
        pm_hiddens.append(bpm) # 파라미터 추가
        if output_shape: # 이전에 output_shape 가 bshape로 초기화 된 경우
            assert output_shape[0:-1] == bshape[0:-1] #채널 수를 제외한 나머지 차원의 크기가 같은지 검사
            output_shape[-1] += bshape[-1] # 해당 분기의 채널 수를 추가함
        else: # 아직 output_shape == None인 상태
            output_shape = bshape 
            
    return {'pms': pm_hiddens}, output_shape

def cnn_ext_forward_parallel_layer(self, x, hconfig, pm):
    bys, bauxes, bchns = [], [], []
    for n, bconfig in enumerate(hconfig[2:]):
        by, baux = self.forward_layer(x, bconfig, pm['pms'][n]) # pm가 딕셔너리 구조고, 'pms'키의 값은 각 분기의 파라미터들을 담는 리스트
        bys.append(by)
        bauxes.append(baux)
        bchns.append(by.shape[-1]) # 각 분기의 출력 채널수를 bchns 리스트에 모음
        
    y = np.concatenate(bys, axis = -1) # bys에 담긴 분기의 출력들을 마지막 축(채널) 기준으로 한데 모아 출력 y를 생성하고 보고
    return y, [bauxes, bchns] # y와 역전파용 보조 정보인 bauxes, bchns를 함께 반환

def cnn_ext_backprop_parallel_layer(self, G_y, hconfig, pm, aux):
    bauxes, bchns = aux
    bcn_from = 0
    G_x = 0
    for n, bconfig in enumerate(hconfig[2:]):
        bcn_to = bcn_from + bchns[n] 
        G_y_slice = G_y[:, :, :, bcn_from:bcn_to] # 이러면 bcn_from이 계속 0이어서 앞부분이 반복해서 나오는 것이 아닌지??
        G_x += self.backprop_layer(G_y_slice, bconfig, pm['pms'][n], bauxes[n]) # 각 분기는 순전파에서 같은 입력을 공유하므로 합산
    return G_x

CnnExtModel.alloc_parallel_layer = cnn_ext_alloc_parallel
CnnExtModel.forward_parallel_layer = cnn_ext_forward_parallel_layer
CnnExtModel.backprop_parallel_layer = cnn_ext_backprop_parallel_layer